# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
output_data_file = "./citydata_df.csv"
citydata_df = pd.read_csv(output_data_file)
citydata_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Ocós,GT,3,2022-04-25,66,14.5094,-92.1933,84.38,5.14
1,Mataura,NZ,90,2022-04-25,76,-46.1927,168.8643,54.48,12.10
2,George Town,MY,20,2022-04-25,79,5.4112,100.3354,84.13,0.00
3,Tarkwa,GH,100,2022-04-25,91,5.3064,-1.9847,72.81,2.73
4,Olinda,BR,75,2022-04-25,74,-8.0089,-34.8553,82.13,9.22
...,...,...,...,...,...,...,...,...,...
570,Batagay-Alyta,RU,100,2022-04-25,82,67.8006,130.4114,14.20,1.07
571,Kwang Binh,VN,7,2022-04-25,72,17.4833,106.6000,84.09,5.35
572,São Sebastião,BR,79,2022-04-25,89,-23.7600,-45.4097,72.50,7.81
573,Caarapó,BR,3,2022-04-25,71,-22.6342,-54.8222,73.00,6.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Store country in locations
locations = citydata_df[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humidity = citydata_df["Humidity"]

In [19]:
fig = gmaps.figure()
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust your specifications, but make sure to limit the number of rows returned by your API requests to a reasonable number.
myweather_df=citydata_df.copy()
myweather_df = myweather_df.loc[(citydata_df['Max Temp'] <= 90) 
                                 & (citydata_df['Max Temp'] > 75) 
                                 & (citydata_df['Wind Speed'] < 15) 
                                 & (citydata_df['Cloudiness']==0) 
                                 & (citydata_df['Humidity'] < 65)]
myweather_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
38,Hamilton,US,0,2022-04-25,54,39.1834,-84.5333,77.09,8.05
121,Atar,MR,0,2022-04-25,32,20.5169,-13.0499,76.32,7.61
138,Cabo San Lucas,MX,0,2022-04-25,63,22.8909,-109.9124,78.62,11.50
151,Ixtlán del Río,MX,0,2022-04-25,28,21.0333,-104.3667,81.41,7.65
203,Pozo Colorado,PY,0,2022-04-25,48,-23.4923,-58.7972,86.54,13.58
253,Huejuquilla el Alto,MX,0,2022-04-25,19,22.6000,-103.8667,75.97,14.09
392,Arlit,NE,0,2022-04-25,11,18.7369,7.3853,83.25,5.88
440,Sur,OM,0,2022-04-25,43,22.5667,59.5289,84.25,9.66
459,Bilma,NE,0,2022-04-25,9,18.6853,12.9164,88.52,6.93
475,Sankheda,IN,0,2022-04-25,30,22.1667,73.5833,76.78,4.07


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
# Store into variable named hotel_df.
hotel_df=myweather_df
# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
38,Hamilton,US,0,2022-04-25,54,39.1834,-84.5333,77.09,8.05,
121,Atar,MR,0,2022-04-25,32,20.5169,-13.0499,76.32,7.61,
138,Cabo San Lucas,MX,0,2022-04-25,63,22.8909,-109.9124,78.62,11.50,
151,Ixtlán del Río,MX,0,2022-04-25,28,21.0333,-104.3667,81.41,7.65,
203,Pozo Colorado,PY,0,2022-04-25,48,-23.4923,-58.7972,86.54,13.58,
253,Huejuquilla el Alto,MX,0,2022-04-25,19,22.6000,-103.8667,75.97,14.09,
392,Arlit,NE,0,2022-04-25,11,18.7369,7.3853,83.25,5.88,
440,Sur,OM,0,2022-04-25,43,22.5667,59.5289,84.25,9.66,
459,Bilma,NE,0,2022-04-25,9,18.6853,12.9164,88.52,6.93,
475,Sankheda,IN,0,2022-04-25,30,22.1667,73.5833,76.78,4.07,


In [22]:
#reset index for loop
hotel_df.reset_index()

,index,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,38,Hamilton,US,0,2022-04-25,54,39.1834,-84.5333,77.09,8.05,
1,121,Atar,MR,0,2022-04-25,32,20.5169,-13.0499,76.32,7.61,
2,138,Cabo San Lucas,MX,0,2022-04-25,63,22.8909,-109.9124,78.62,11.50,
3,151,Ixtlán del Río,MX,0,2022-04-25,28,21.0333,-104.3667,81.41,7.65,
4,203,Pozo Colorado,PY,0,2022-04-25,48,-23.4923,-58.7972,86.54,13.58,
5,253,Huejuquilla el Alto,MX,0,2022-04-25,19,22.6000,-103.8667,75.97,14.09,
6,392,Arlit,NE,0,2022-04-25,11,18.7369,7.3853,83.25,5.88,
7,440,Sur,OM,0,2022-04-25,43,22.5667,59.5289,84.25,9.66,
8,459,Bilma,NE,0,2022-04-25,9,18.6853,12.9164,88.52,6.93,
9,475,Sankheda,IN,0,2022-04-25,30,22.1667,73.5833,76.78,4.07,


In [23]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
hotel_name = []
for row in range(len(hotel_df)):
    lat = hotel_df.iloc[row]['Latitude']
    long = hotel_df.iloc[row]['Longitude']
    latlong = f'{lat},{long}'
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latlong}&radius=5000&type=hotel&key={g_key}'
    response = requests.get(url).json()
    
    hotel_name.append(response['results'][1]['name'])
hotel_df['Hotel Name'] = hotel_name
hotel_df.head()

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
38,Hamilton,US,0,2022-04-25,54,39.1834,-84.5333,77.09,8.05,WSTR-TV (Cincinnati)
121,Atar,MR,0,2022-04-25,32,20.5169,-13.0499,76.32,7.61,Oumou Elghoura
138,Cabo San Lucas,MX,0,2022-04-25,63,22.8909,-109.9124,78.62,11.50,Hotel Tesoro Los Cabos
151,Ixtlán del Río,MX,0,2022-04-25,28,21.0333,-104.3667,81.41,7.65,servicio postal mexicano
203,Pozo Colorado,PY,0,2022-04-25,48,-23.4923,-58.7972,86.54,13.58,Electro chaco Martinez y cia


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer ontop of heat map
hotel_marker=gmaps.marker_layer(locations, hotel_info)
# Display figure
fig.add_layer(heat_layer)
fig.add_layer(hotel_marker)

fig

Figure(layout=FigureLayout(height='420px'))